<a href="https://colab.research.google.com/github/vissa1604/gitTraining/blob/master/bibliography_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
import numpy as np
import pandas as pd
from google.colab import drive

### Mounting drive to read the training files

In [ ]:
drive.mount('/content/drive')

In [ ]:
!ls "/content/drive/My Drive"

In [ ]:
%cd /content/drive/My Drive/bibliography_identification/

In [ ]:
df = pd.read_excel('citationlist.xlsx')

In [ ]:
df = df.sample(n=10000)

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
def clean_string(data):                                                       
  try:
    text_data = data
    text_data = re.sub(r'^https?:\/\/<>.*[\r\n]*','',text_data,flags=re.MULTILINE) 
    text_data= re.sub(r"[^A-Za-z]"," ",text_data)
    text_data = text_data.replace("\n","").strip()
    return text_data
  except:
    return data

In [ ]:
cleaned_data = []
for text in df['References'].values:
  cleaned_text = clean_string(text)
  cleaned_data.append(cleaned_text)



In [ ]:
import tensorflow_hub as hub

In [ ]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [ ]:
cleaned_data = [str(data) for data in cleaned_data]

In [ ]:
embeddings = embed(cleaned_data)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [ ]:
vector = TfidfVectorizer()

In [ ]:
X = vector.fit_transform(cleaned_data)
X.shape

In [ ]:
vect = X.toarray()

In [ ]:
df['References'] = vect

In [ ]:
X = np.array(df['References']).reshape(-1, 1)
Y = df['Citation']

In [ ]:
X.shape, Y.shape

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
alg = LogisticRegression()

In [ ]:
alg.fit(X_train, Y_train)

In [ ]:
y_predict = alg.predict(X_test)

In [ ]:
alg.score(X_train, Y_train)

In [ ]:
alg.score(X_test, Y_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 100)
classifier.fit(X_train, Y_train)

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
train_score = classifier.score(X_train, Y_train)
test_score = classifier.score(X_test, Y_test)

train_score, test_score